In [1]:
import glob
from PIL import Image
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import requests
import json

In [2]:
#display all columns (not just first and last 5)
pd.options.display.max_columns = None

In [3]:
data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
data = json.loads(data.content)

for key in data:
    print (key)

chips
events
game_settings
game_config
phases
teams
total_players
element_stats
element_types
elements


# Teams

In [4]:
len(data['teams'])

20

In [5]:
team_df = pd.DataFrame(data['teams'])
team_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,2,ARS,5,None,False,0,1350,1350,1390,1400,1310,1300,1
1,7,0,None,2,0,Aston Villa,0,0,6,AVL,3,None,False,0,1145,1240,1130,1180,1160,1300,2
2,91,0,None,3,0,Bournemouth,0,0,9,BOU,3,None,False,0,1170,1200,1120,1180,1220,1220,127
3,94,0,None,4,0,Brentford,0,0,10,BRE,3,None,False,0,1130,1180,1100,1100,1160,1260,130
4,36,0,None,5,0,Brighton,0,0,8,BHA,3,None,False,0,1140,1165,1090,1140,1190,1190,131


# Players Season Cumulative Stats

In [6]:
player_season_df = pd.DataFrame(data['elements'])
player_season_df.sort_values('minutes', ascending=False).head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,birth_date,has_temporary_code,opta_code,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,mng_win,mng_draw,mng_loss,mng_underdog_win,mng_underdog_draw,mng_clean_sheets,mng_goals_scored,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
272,True,False,100.0,100.0,172649,0,0,1,-1,3,1,2.3,2.0,2,Dean,3.0,201,False,,2025-01-12T22:00:08.068713Z,46,172649.jpg,3.6,False,Henderson,11.7,False,None,a,7,31,135,1875205,0,2293286,0,0.7,29.3,Henderson,241.0,2023-08-31,1997-03-12,False,p172649,3420,0,0,11,51,0,1,0,3,0,104,8,466,853.6,30.0,4.0,88.6,38,0.01,0.02,0.03,50.33,0,0,0,0,0,0,0,20,6,406,4,465,2,172,6,NaN,,NaN,,NaN,,0.00,2.74,0.00,0.00,1.32,1.34,352,13,125,12,78,13,36,5,1.0,0.29
346,True,False,NaN,NaN,80201,0,0,0,0,1,1,2.5,2.5,2,Bernd,3.0,248,False,,None,50,80201.jpg,3.0,False,Leno,2.9,False,None,a,9,54,115,594796,0,587626,0,0.6,23.0,Leno,80.0,2022-08-02,1992-03-04,False,p80201,3420,0,2,5,54,0,1,0,5,0,110,5,475,967.0,20.0,0.0,98.6,38,0.00,0.03,0.03,47.75,0,0,0,0,0,0,0,10,4,432,8,770,74,149,3,NaN,,NaN,,NaN,,0.00,2.89,0.00,0.00,1.26,1.42,195,7,128,13,142,21,114,17,1.0,0.13
124,True,False,NaN,NaN,432830,0,0,1,-1,2,2,5.2,4.6,2,Nathan,4.6,88,False,,None,46,432830.jpg,3.3,False,Collins,4.5,False,None,a,4,94,127,992798,0,568992,0,1.0,27.6,Collins,104.0,2023-07-04,2001-04-30,False,p432830,3420,2,7,8,57,0,0,0,5,0,0,11,549,1023.2,160.5,311.0,149.6,38,2.44,1.57,4.01,56.31,0,0,0,0,0,0,0,7,1,239,61,106,7,77,10,NaN,,NaN,,NaN,,0.06,0.00,0.04,0.10,1.48,1.50,357,52,46,11,114,26,83,28,1.0,0.21
621,True,False,100.0,100.0,85633,0,0,7,-7,3,1,2.3,2.3,2,Matz,1.8,443,False,,2024-10-12T11:00:07.468908Z,52,85633.jpg,3.9,False,Sels,19.9,False,None,a,16,17,150,3541533,0,1554132,0,0.3,28.8,Sels,21.0,2024-02-01,1992-02-26,False,p85633,3420,0,1,13,46,0,0,0,4,0,120,11,521,942.2,20.2,0.0,96.7,38,0.00,0.26,0.26,50.17,0,0,0,0,0,0,0,12,5,430,6,785,77,154,5,NaN,,NaN,,NaN,,0.00,3.16,0.01,0.01,1.32,1.21,161,4,218,18,51,9,17,2,1.0,0.34
316,True,False,NaN,NaN,111234,0,0,2,-2,5,1,3.3,5.0,11,Jordan,5.5,235,True,,None,52,111234.jpg,4.2,False,Pickford,15.9,False,None,a,8,11,158,2522034,0,2440525,0,1.1,30.4,Pickford,241.0,2017-07-01,1994-03-07,False,p111234,3420,0,1,12,44,0,2,0,7,0,122,15,593,1052.0,20.3,0.0,107.0,38,0.00,0.22,0.22,46.11,0,0,0,0,0,0,0,5,2,429,5,787,79,140,2,NaN,,NaN,,NaN,,0.00,3.21,0.01,0.01,1.21,1.16,165,6,30,2,32,6,24,3,1.0,0.32


In [7]:
#get list of IDs to scrape by GW stats per player
player_id_list = list(player_season_df[['id']].drop_duplicates().id)
assert len(player_id_list)== len(player_season_df)

# Players By Gameweek Stats

In [8]:
player_weekly_df = pd.DataFrame()

for pid in player_id_list:
    #print(pid)
    response = requests.get(f'https://fantasy.premierleague.com/api/element-summary/{pid}/')
    
    if response.status_code != 200:
        raise Exception("Response was code " + str(response.status_code))
        print(pid)
        
    p_data = pd.DataFrame(response.json()['history'])
    p_data['id'] = pid
    player_weekly_df = player_weekly_df.append(p_data)
    


In [9]:
assert len(player_weekly_df[['element']].drop_duplicates())== len(player_season_df)

In [10]:
#merge on player name and team name to player_weekly_df
player_names = player_season_df[['id', 'team', 'element_type', 'first_name', 'second_name', 'web_name']].drop_duplicates()
team_names = team_df[['id', 'name', 'short_name']].drop_duplicates().rename(columns={'id':'team'})
opponent_names = team_df[['id', 'name', 'short_name']].drop_duplicates().rename(columns={'id':'opponent_id', 'name':'opponent', 'short_name':'opponent_short_name'})


player_weekly_with_deets = (
    player_weekly_df
    .merge(player_names, left_on='id', right_on='id', how='left')
    .merge(team_names, left_on='team', right_on='team', how='left')
    .merge(opponent_names, left_on='opponent_team', right_on='opponent_id', how='left')
    .rename(columns={
        'team' : 'team_id',
        'name' : 'team'
    })
)

for var in [
    'influence', 'creativity', 'threat', 'ict_index', 'starts', 
    'expected_goals', 'expected_assists', 'expected_goal_involvements',
    'expected_goals_conceded'
]:
    player_weekly_with_deets[var] = player_weekly_with_deets[var].astype(float)

player_weekly_with_deets.loc[(player_weekly_with_deets['element_type'] == 1), 'position'] = 'GK'
player_weekly_with_deets.loc[(player_weekly_with_deets['element_type'] == 2), 'position'] = 'DEF'
player_weekly_with_deets.loc[(player_weekly_with_deets['element_type'] == 3), 'position'] = 'MID'
player_weekly_with_deets.loc[(player_weekly_with_deets['element_type'] == 4), 'position'] = 'FWD'

In [11]:
player_weekly_with_deets.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,mng_win,mng_draw,mng_loss,mng_underdog_win,mng_underdog_draw,mng_clean_sheets,mng_goals_scored,value,transfers_balance,selected,transfers_in,transfers_out,id,team_id,element_type,first_name,second_name,web_name,team,short_name,opponent_id,opponent,opponent_short_name,position
0,1,2,20,0,True,2024-08-17T14:00:00Z,2,0,1,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,55,0,2923,0,0,1,1,3,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,ARS,20,Wolves,WOL,MID
1,1,11,2,0,False,2024-08-24T16:30:00Z,0,2,2,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,55,-790,2321,84,874,1,1,3,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,ARS,2,Aston Villa,AVL,MID
2,1,21,5,0,True,2024-08-31T11:30:00Z,1,1,3,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,54,-279,2397,355,634,1,1,3,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,ARS,5,Brighton,BHA,MID
3,1,39,18,0,False,2024-09-15T13:00:00Z,0,1,4,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,54,-747,1650,0,747,1,1,3,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,ARS,18,Spurs,TOT,MID
4,1,47,13,0,False,2024-09-22T15:30:00Z,2,2,5,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,54,-174,1494,0,174,1,1,3,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,ARS,13,Man City,MCI,MID


# Fixtures

In [12]:
response = requests.get("https://fantasy.premierleague.com/api/fixtures/")
fixtures_df = pd.DataFrame(response.json())
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2444470,1,True,True,1,2024-08-16T19:00:00Z,90,False,True,9,0,14,1,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,3,115827
1,2444473,1,True,True,4,2024-08-17T11:30:00Z,90,False,True,12,2,10,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,115830
2,2444471,1,True,True,2,2024-08-17T14:00:00Z,90,False,True,20,0,1,2,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",3,5,115828
3,2444472,1,True,True,3,2024-08-17T14:00:00Z,90,False,True,5,3,8,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,3,115829
4,2444474,1,True,True,5,2024-08-17T14:00:00Z,90,False,True,17,0,15,1,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",1,4,115831


# Export

In [13]:
#export to CSV
fpl_dir = '/Users/yuta/Desktop/fpl_web_scrape/2024-2025'

team_df.to_csv(f"{fpl_dir}/teams.csv", index=False)
fixtures_df.to_csv(f"{fpl_dir}/fixtures.csv", index=False)
player_season_df.to_csv(f"{fpl_dir}/player_season_stats.csv", index=False)
#player_weekly_df.to_csv(f"{fpl_dir}/data/player_weekly_stats.csv", index=False)
player_weekly_with_deets.to_csv(f"{fpl_dir}/player_weekly_stats.csv", index=False)